In [3]:
import pandas as pd

In [ ]:
def interpolate_rates(county, veh_type_list, initial_year, final_year, max_ev_fraction=1.0):
       """
       Generate linear trend from initial_year to final_year for EV registrations. 
       EVs will make up the max_ev_fractionp of registrations in final_year, scaled from the initial_year value.
       The remaining vehicle types are scaled proportionally down to values in final year of trend
       
       Args:
           county (str): The name of the county.
           veh_type_list (list): sourceTypeID of vehicle types to process.
           initial_year (int): The starting year for interpolation to final
           final_year (int): The ending year for interpolation.
           max_ev_fraction (float): The maximum annual registration share of EVs (plateau)
       """

       # Load the original AVFT data to adjust future values
       df = pd.read_csv(f'Y:/Air Quality/2026_2050_RTP/wa_dept_of_ecology_data/2023_MOVES_inputs_revised/{county}/{county}_avft_2023.csv')

       # Loop through each vehicle type
       for veh_type in veh_type_list:
              
              # Select EVs (fuelTypeID = 9) and the specified sourceTypeID
              filter = (
                     (df['fuelTypeID'] == 9) & 
                     (df['modelYearID']==initial_year) & 
                     (df['sourceTypeID']==veh_type)
              )
              
              if len(df.loc[filter,'fuelEngFraction'].values) == 1:
                     # Get the initial value for fuelEngFraction
                     initial_value = df.loc[filter,'fuelEngFraction'].values[0]

              # Get the shares of EVs in initial_year
              initial_value = df.loc[(df['fuelTypeID'] == 9) & 
                     (df['modelYearID']==initial_year) & 
                     (df['sourceTypeID']==veh_type),'fuelEngFraction'].values[0]

              # Fill in intermediate years for EVs and other fuel types
              for year in range(initial_year, final_year+1):
                     # Calculate the scale factor for the current year
                     scale_factor = (year - initial_year) / (final_year - initial_year)

                     # Get the actual original value for fuelEngFraction (the "initial_value" field is only used for linear interpolation)
                     original_value = df.loc[(df['fuelTypeID'] == 9) & 
                            (df['modelYearID']==year) & 
                            (df['sourceTypeID']==veh_type),'fuelEngFraction'].values[0]

                     # Calculate the new value for fuelEngFraction for EVs
                     new_value = initial_value * (1 - scale_factor) + max_ev_fraction * scale_factor
                     # Update the DataFrame for the current year and sourceTypeID 21
                     df.loc[(df['fuelTypeID'] == 9) & 
                            (df['modelYearID'] == int(year)) & 
                            (df['sourceTypeID']==veh_type),'fuelEngFraction'] = new_value
                     

                     # Reduce fuelEngFraction for other fuel types proportinally
                     other_filter = (
                            (df['fuelTypeID'] != 9) & 
                            (df['modelYearID'] == int(year)) & 
                            (df['sourceTypeID']==veh_type)
                     )
                     other_frac = df.loc[other_filter,'fuelEngFraction']
                     
                     new_vals = other_frac - (other_frac/other_frac.sum())*(new_value-original_value)
                     df.loc[other_filter,'fuelEngFraction'] = new_vals

              # Fill in remaining years past final_year with the same values as in final year
       
              for year in range(final_year+1, df['modelYearID'].max()):
                     # Update the DataFrame for the current year and sourceTypeID 21
                     df.loc[(df['fuelTypeID'] == 9) & 
                            (df['modelYearID']>= int(final_year)) & 
                            (df['sourceTypeID']==veh_type),'fuelEngFraction'] = max_ev_fraction
                     
                     for fuel_type in [1,2,3,5]:
                            # Reduce fuelEngFraction for other fuel types proportinally
                            other_filter = 
                            fuel_type_frac = df.loc[(
                                   (df['fuelTypeID'] == fuel_type) & 
                                   (df['modelYearID'] == int(final_year)) & 
                                   (df['sourceTypeID']==veh_type)
                            ),'fuelEngFraction']
                            df.loc[(
                                   (df['fuelTypeID'] == fuel_type) & 
                                   (df['modelYearID'] >= int(final_year)) & 
                                   (df['sourceTypeID']==veh_type)
                            ), 'fuelEngFraction'] = fuel_type_frac
                            # [other_filter,'fuelEngFraction'] = new_vals
                     # df.loc[(df['fuelTypeID'] != 9) & 
                     #        (df['modelYearID']>= int(final_year)) & 
                     #        (df['sourceTypeID']==veh_type),'fuelEngFraction'] = 0
       
       return df

In [9]:
# Create a scenario where all light duty vehicles are electric by 2035
# Scale fuelTypeID up to 1 and all other values down to 0
# Only apply to light-duty vehicles (sourceTypeID 21, 31, 32)
veh_type_list = [21,31,32]

# Process these one sourceTypeID at a time
# initial_year is when trend begins and final_year is when 100% EV sales are achieved
initial_year = 2026
final_year = 2035

county = 'King'
df = interpolate_rates(county, veh_type_list, initial_year, final_year, max_ev_fraction=0.65)
# df.to_csv(f'Y:/Air Quality/2026_2050_RTP/scenarios/2040_full_light_duty_EV/2023_MOVES_inputs_revised/{county}/{county}_avft_2023.csv', index=False)

KeyboardInterrupt: 

In [ ]:
# Process medium and heavy duty vehicles
# df
# Per Washington’s Clean Trucks rules, by 2035 40-75% of medium and heavy-duty truck sales must be ZEV:  
#  55% for Class 2b-3 8.5-14k lbs; 75% for Class 4-8 14-60k lbs; 40% for Class 7-8 26-60k lbs. 
# Assume they plateau at final year of implementation

# Single Unit Trucks are considered medium 
# These range from class 5-7 
# 75% for class 4-8, >14



# Combination (short and long haul) are considered heavy duty
# There are class 7-8 and above according to FHWA vehicle categories
# 40% target for class 7-8, 26-60k tractors, plateaus at 2032...

# 55% for class 2b-3, 8.5-14k


,sourceTypeID,modelYearID,fuelTypeID,engTechID,fuelEngFraction
0,11,1960,1,1,1.000000
1,11,1961,1,1,1.000000
2,11,1962,1,1,1.000000
3,11,1963,1,1,1.000000
4,11,1964,1,1,1.000000
...,...,...,...,...,...
3732,62,2059,9,40,0.045774
3733,62,2060,2,1,0.941490
3734,62,2060,3,1,0.012737
3735,62,2060,9,30,0.000000
